In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline

%load_ext autoreload
%autoreload 2

folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure4_voyages'
figure_folder = '{}/from_multimodal_to_extremes/motifs'.format(folder)
! mkdir $figure_folder


mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure4_voyages/from_multimodal_to_extremes/motifs': File exists


/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


## Count kmers

In [3]:
bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/multimodal_to_extremes'

In [4]:
cd $bed_folder

/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure4_voyages/multimodal_to_extremes


In [5]:
import glob

genome = '/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa'

for bed in glob.glob('*placental*bed'.format(bed_folder)):
    print(bed)
    fasta = bed.replace('.bed', '.fasta')
    ! bedtools getfasta -s -bed $bed -fi $genome -fo $fasta

exon2_placental_background.bed
exon2_downstream400_placental_foreground.bed
exon2_upstream400_placental_foreground.bed
exon2_placental_foreground.bed
exon2_downstream400_placental_background.bed
exon2_upstream400_placental_background.bed


In [6]:
ll *fasta

-rw-r--r-- 1 obotvinnik   3840 Jan 11 10:11 aligned.fasta
-rw-r--r-- 1 obotvinnik 215386 Feb 23 15:34 exon2_downstream400_placental_background.fasta
-rw-r--r-- 1 obotvinnik   2722 Feb 23 15:34 exon2_downstream400_placental_foreground.fasta
-rw-r--r-- 1 obotvinnik 344535 Feb 23 15:34 exon2_placental_background.fasta
-rw-r--r-- 1 obotvinnik   2170 Feb 23 15:34 exon2_placental_foreground.fasta
-rw-r--r-- 1 obotvinnik 217873 Feb 23 15:34 exon2_upstream400_placental_background.fasta
-rw-r--r-- 1 obotvinnik   2514 Feb 23 15:34 exon2_upstream400_placental_foreground.fasta
-rw-r--r-- 1 obotvinnik   2605 Jan 11 11:53 kmer2.fasta
-rw-r--r-- 1 obotvinnik   1565 Jan 11 09:31 temp.fasta
-rw-r--r-- 1 obotvinnik   3185 Jan 11 12:03 top_kmers_downstream400.aligned.fasta
-rw-r--r-- 1 obotvinnik   1981 Jan 11 12:28 top_kmers_downstream400.fasta
-rw-r--r-- 1 obotvinnik   4064 Jan 11 12:03 top_kmers_exonbody.aligned.fasta
-rw-r--r-- 1 obotvinnik   2641 Jan 11 12:28 top_kmers_exonbody.fasta
-rw-r--r-- 1 ob

In [7]:
import kvector
from collections import defaultdict

In [8]:
from kvector.kmer import count_kmers

In [10]:
kmer_matrices = defaultdict(dict)

for fasta in glob.glob('exon2*fasta'):
    print(fasta)
    prefix = fasta.split('.')[0]
    split = prefix.split('_')
    if 'stream' in prefix:
        region = split[1]
    else:
        region = 'exonbody'
    x_ground = split[-1]
    kmer_matrix = count_kmers(fasta)
    kmer_matrices[region][x_ground] = kmer_matrix

exon2_upstream400_placental_foreground.fasta
exon2_placental_foreground.fasta
exon2_upstream400_placental_background.fasta
exon2_placental_background.fasta
exon2_downstream400_placental_foreground.fasta
exon2_downstream400_placental_background.fasta


In [ ]:
foreground = kmer_matrices['upstream400']['foreground']

In [ ]:
kmer_zscores = pd.DataFrame()

for region, matrices in kmer_matrices.items():
    foreground = matrices['foreground']
    
    # "background" from homer didn't include the foreground events but we need to for this analysis
    background = pd.concat([matrices['background'], matrices['foreground']])
    zscore = (foreground.mean() - background.mean())/background.var()
    kmer_zscores[region] = zscore
kmer_zscores.head()

In [ ]:
kmer_zscores.sort_values(by='upstream400', ascending=False).head(20)

In [ ]:
kmer_zscores.sort_values(by='downstream400', ascending=False).head(20)

In [ ]:
kmer_zscores_top = kmer_zscores[kmer_zscores > (kmer_zscores.mean() + 2*kmer_zscores.std())].dropna(how='all')
print(kmer_zscores_top.shape)
kmer_zscores_top.head()

### Align top kmers

In [ ]:
! clustalo --help

In [ ]:
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_dna
from Bio import SeqIO

In [ ]:
!muscle -h

In [ ]:
# for region, col in kmer_zscores_top.iteritems():

region = 'upstream400'
col = kmer_zscores_top[region]
print '\n\n', region
col = col.dropna()

kmers = col.index

records = kmers.map(lambda x: SeqRecord(Seq(x, alphabet=generic_dna), id=x, name='', description=''))

fasta = 'top_kmers_{}.fasta'.format(region)
with open(fasta, 'w') as f:
    SeqIO.write(records, f, 'fasta')


alignment_scores = pd.DataFrame(index=kmers, columns=kmers, dtype=float)

for seq1, seq2 in itertools.combinations(kmers, 2):
    alignment_scores.loc[seq1, seq1] = 0
    alignment_scores.loc[seq2, seq2] = 0
    try:
        # Get minimum matching
        aligned = min(pairwise2.align.localxx(seq1, seq2), key=lambda x: x[2])
        seq1_aligned, seq2_aligned, score, begin, end = aligned
        distance = length - score
    except ValueError:
        distance = max(len(seq1), len(seq2))
    alignment_scores.loc[seq1, seq2] = score
    alignment_scores.loc[seq2, seq1] = score
# alignment_scores.head()

aligned = fasta.replace('.fasta', '.aligned.fasta')
distance_matrix = fasta.replace('.fasta', '.distancematrix')
guidetree = aligned.replace('.fasta', '.guidetree')

alignment_scores.to_csv(distance_matrix, sep='\t', header=False)
n_kmers = len(kmers)
! echo $n_kmers | cat - $distance_matrix > distance_matrix

! clustalo --infile $fasta --outfile $aligned --full --force --guidetree-out=$guidetree --seqtype=DNA --distmat-in=$distance_matrix

with open(aligned) as f:
    records_aligned = AlignIO.read(aligned, 'fasta')
# for record in records_aligned: 
#     print str(record.seq)

tree = Phylo.read(guidetree, 'newick')
tree.ladderize()
fig, ax = plt.subplots(figsize=(4, 20))
Phylo.draw(tree, axes=ax)
#     sns.despine(fig=fig, bottom=True, left=True)

In [ ]:
for region, col in kmer_zscores_top.iteritems():

    print '\n\n', region
    col = col.dropna()

    kmers = col.index

    records = kmers.map(lambda x: SeqRecord(Seq(x, alphabet=generic_dna), id=x, name='', description=''))

    fasta = 'top_kmers_{}.fasta'.format(region)
    with open(fasta, 'w') as f:
        SeqIO.write(records, f, 'fasta')

    ! clustalo --infile $fasta --outfile $aligned --full --force --guidetree-out=$guidetree --seqtype=DNA

    with open(aligned) as f:
        records_aligned = AlignIO.read(aligned, 'fasta')
    # for record in records_aligned: 
    #     print str(record.seq)

    tree = Phylo.read(guidetree, 'newick')
    tree.ladderize()
    fig, ax = plt.subplots(figsize=(4, 20))
    Phylo.draw(tree, axes=ax)
    #     sns.despine(fig=fig, bottom=True, left=True)

In [ ]:
import itertools
 
def terminal_neighbor_dists(tree):
    """Return a list of distances between adjacent terminals."""
    def generate_pairs(tree):
        pairs = itertools.tee(tree)
        pairs[1].next()
        return itertools.izip(pairs[0], pairs[1])
    return [tree.distance(*i) for i in
            generate_pairs(tree.find_clades(terminal=True))]

len(terminal_neighbor_dists(tree))

In [ ]:
tree.terminal_neighbor_dists()

In [ ]:
list(tree.find_clades())

In [ ]:
depths = tree.depths()
depths

In [ ]:
! pip install fisher

In [ ]:
! pip install ete2

In [ ]:
treecut = guidetree  + ".treecut"

In [ ]:
! conda install --yes rpy2

In [ ]:
! python /home/obotvinnik/workspace-git/treecut/treecut.py

In [ ]:
! python /home/obotvinnik/workspace-git/treecut/treecut.py $guidetree $treecut

In [ ]:
tree.depths()

In [ ]:
len(list(tree.find_clades()))

In [ ]:
import matplotlib as mpl

deep = map(mpl.colors.rgb2hex, sns.color_palette('deep'))
deep

In [ ]:
sns.palplot(deep)

In [ ]:
# Styling ACGT on tables in colors jQuery code. Must be in this order because otherwise you're replacing the "G" in "Green" or the "C" in "Crimson" instead of the bare letter

/* G = yellow */
$('table').each(function () {
    $(this).html($(this).html().replace(/(G)/g, '<span style="color: #ccb974;">$1</span>'));
});

/* A = green */
$('table').each(function () {
    $(this).html($(this).html().replace(/(A)/g, '<span style="color: #55a868;">$1</span>'));
});

/* C = blue */
$('table').each(function () {
    $(this).html($(this).html().replace(/(C)/g, '<span style="color: #4c72b0;">$1</span>'));
});

/* T = red */ 
$('table').each(function () {
    $(this).html($(this).html().replace(/(T)/g, '<span style="color: #c44e52;">U</span>'));
});

In [ ]:
top_upstream_kmers = kmer_zscores_top['upstream400'].dropna()
top_upstream_kmers.to_frame().head()

In [ ]:
from Bio import pairwise2

In [ ]:
import itertools

In [ ]:
alignment = pairwise2.align.localxx('CGACT', 'ATCGA')[0]
for x in alignment:
    print(x)

Output from alignment:

    seq1_aligned, seq2_aligned, score, begin, end = alignment

In [ ]:
alignment = pairwise2.align.globalxx('CGACT', 'ATCGA')[0]
for x in alignment:
    print(x)

In [ ]:
pairwise2??

In [ ]:
alignment_scores = pd.DataFrame(index=top_upstream_kmers.index, columns=top_upstream_kmers.index)

for seq1, seq2 in itertools.combinations(top_upstream_kmers.index, 2):
    alignment_scores.loc[seq1, seq1] = 1.0
    alignment_scores.loc[seq2, seq2] = 1.0
    try:
        # Get minimum matching
        aligned = min(pairwise2.align.localxx(seq1, seq2), key=lambda x: x[2])
        seq1_aligned, seq2_aligned, score, begin, end = aligned
        distance = length - score
    except ValueError:
        distance = max(len(seq1), len(seq2))
    alignment_scores.loc[seq1, seq2] = score
    alignment_scores.loc[seq2, seq1] = score
alignment_scores.head()

In [ ]:
length- score

In [ ]:
alignment_scores_tidy = alignment_scores.unstack().reset_index()
alignment_scores_tidy = alignment_scores_tidy.rename(columns={'level_0': "kmer1", 'level_1': 'kmer2', 
                                                              0: 'local_alignment_score'})
alignment_scores_tidy.head()

In [ ]:
alignment_scores_tidy.shape

In [ ]:
alignment_scores_tidy = alignment_scores_tidy.loc[alignment_scores_tidy.local_alignment_score >= 2]
print alignment_scores_tidy.shape
alignment_scores_tidy.head()

In [ ]:
kmer_df = alignment_scores_tidy.groupby('kmer1').get_group('ATCGA')
kmer_df.head()

In [ ]:
from Bio.Align.Applications import ClustalOmegaCommandline

In [ ]:
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_dna
from Bio import SeqIO

In [ ]:
records = kmer_df.kmer2.map(lambda x: SeqRecord(Seq(x, alphabet=generic_dna), id=x, name='', description=''))

In [ ]:
fasta = 'kmer2.fasta'
with open(fasta, 'w') as f:
    SeqIO.write(records, f, 'fasta')

In [ ]:
records.iloc[0]

In [ ]:
! makeblastdb -in kmer2.fasta -out kmer2 -dbtype nucl -title "kmer2" 

In [ ]:
! blastn -h

In [ ]:
%%bash 

blastn -db kmer2 -query kmer2.fasta -outfmt 6 -out kmer2.blast.txt -evalue 100
head kmer2.blast.txt

In [ ]:
! 

In [ ]:
from Bio import SearchIO

In [ ]:
Applications.

In [ ]:
aligned = 'aligned.fasta'

clustalomega_cline = ClustalOmegaCommandline(infile=fasta, outfile=aligned, auto=True, force=True, fulliter=True)
stdout, stderr = clustalomega_cline()

In [ ]:
guidetree = 'aligned.guidetree'

! clustalo --infile $fasta --outfile $aligned --full-iter --force --guidetree-out=$guidetree

In [ ]:
! head aligned.tree

In [ ]:
with open(aligned) as f:
    records_aligned = AlignIO.read(aligned, 'fasta')
for record in records_aligned: 
    print str(record.seq)

In [ ]:
from Bio import Phylo

In [ ]:
sns.set(style='ticks', context='paper')

In [ ]:
tree = Phylo.read(guidetree, 'newick')
tree.ladderize()
fig, ax = plt.subplots(figsize=(4, 30))
Phylo.draw(tree, axes=ax)
sns.despine(fig=fig, bottom=True, left=True)

In [ ]:
from Bio.Phylo.Consensus import strict_consensus, majority_consensus, adam_consensus
trees = list(Phylo.parse(guidetree, 'newick'))

In [ ]:
strict = strict_consensus(trees)
strict.ladderize()
# fig, ax = plt.subplots(figsize=(4, 30))
Phylo.draw(strict)
# sns.despine(fig=fig, bottom=True, left=True)

In [ ]:
majority = majority_consensus(trees)
majority.ladderize()
fig, ax = plt.subplots(figsize=(4, 30))
Phylo.draw(majority, axes=ax)
# sns.despine(fig=fig, bottom=True, left=True)

In [ ]:
adam = adam_consensus(trees)
adam.ladderize()
Phylo.draw(adam)

In [ ]:
alignment_scores = alignment_scores.fillna(0)

In [ ]:
sns.set(context='paper')

In [ ]:
sns.clustermap(alignment_scores, figsize=(25, 25))

In [ ]:
sns.clustermap(alignment_scores, metric='euclidean', method='ward', figsize=(30, 30))

In [ ]:
sns.clustermap(alignment_scores, metric='cityblock', method='average', 
               figsize=(30, 30), cmap='BuGn')

In [ ]:
kmer_zscores0 = kmer_zscores.dropna(how='all').fillna(0)
print(kmer_zscores0.shape)
kmer_zscores0.head()

In [ ]:
kmer_matrices['downstream400']['foreground']['AGTGCG'].sum()

In [ ]:
kmer_matrices['downstream400']['background']['AGTGCG'].sum()

In [ ]:
kmer_zscores.describe()

In [ ]:
sns.pairplot(kmer_zscores.fillna(0))

In [ ]:
g = sns.jointplot('upstream400', 'downstream400', kmer_zscores.fillna(0), size=2.5, 
              annot_kws=dict(loc='upper right'), color='#262626', alpha=0.5)
g.savefig('{}/upstream_vs_downstream_kmer_z_scores.pdf'.format(figure_folder))

## Get motif enrichment of kmers

In [ ]:
cisbp_folder = '/projects/ps-yeolab/obotvinnik/cisbp_v2'
cisbp_kmers = pd.read_csv('{}/kmers_in_cisbp_motifs_thresh05.csv'.format(cisbp_folder), index_col=0)

In [ ]:
cisbp_metadata = pd.DataFrame.from_records(list(cisbp_kmers.columns.map(lambda x: x.split('_'))))
# cisbp_metadata = cisbp_metadata.replace(None, np.nan)
cisbp_metadata.index = cisbp_kmers.columns.map(lambda x: x.split()[0])
cisbp_metadata = cisbp_metadata.rename(columns={0:'Motif ID', 1: "Version", 2: "RBP Name", 3: "RBP ID", 4: "Genus", 5: "Species"})
cisbp_metadata['Full Species'] = cisbp_metadata['Genus'] + ' ' + cisbp_metadata['Species']
cisbp_metadata.head()

In [ ]:
cisbp_metadata.to_csv('{}/cisbp_motif_metadata.csv'.format(cisbp_folder))

In [ ]:
from scipy.stats import ttest_1samp

# kmer_cisbp_ttest = pd.DataFrame(index=cisbp_kmers.columns, columns=study.supplemental.kmer_scores.columns)

lines = []

for intron_name, intron_col in kmer_zscores.iteritems():
    for cisbp_name, cisbp_kmer in cisbp_kmers.iteritems():
#         statistic, p_value = ttest_1samp(intron_col[cisbp_kmer], kmer_means[intron_name])
        statistic, p_value = ttest_1samp(intron_col[cisbp_kmer], 0)
        lines.append([intron_name, cisbp_name, statistic, p_value])
kmer_cisbp_ttest_tidy = pd.DataFrame(lines, columns=['Intron', 'RBP Motif', 'T-Statistic', '$p$-value'])
t_statistic = kmer_cisbp_ttest_tidy.pivot(index='RBP Motif', columns='Intron', values='T-Statistic')
t_statistic.head()

In [ ]:
t_statistic.describe()

In [ ]:
t_statistic_top = t_statistic[t_statistic > (t_statistic.mean() + t_statistic.std())].dropna(how='all')
print(t_statistic_top.shape)
t_statistic_top

In [ ]:
sns.pairplot(t_statistic.fillna(0), size=2)

In [ ]:
t_statistic.sort_values('downstream400').head(10)

In [ ]:
t_statistic.sort_values('downstream400', ascending=False).head(10)

In [ ]:
t_statistic.sort_values('upstream400').head(10)

In [ ]:
t_statistic.sort_values('upstream400', ascending=False).head(10)

In [ ]:
figure_folder

In [ ]:
g = sns.jointplot('upstream400', 'downstream400', t_statistic.fillna(0), size=2.5, 
              annot_kws=dict(loc='lower right'), color='#262626', alpha=0.5)
g.savefig('{}/upstream_vs_downstream_cisbp_t_statistics.pdf'.format(figure_folder))

In [ ]:
# t_statistic_homo_sapiens = t_statistic.loc[cisbp_metadata['Full Species'] == 'Homo sapiens']
# print t_statistic_homo_sapiens.shape
# sns.clustermap(t_statistic_homo_sapiens)